In [185]:
import os
import sys
import h5py
import shutil
import pandas as pd
from astropy.nddata.utils import Cutout2D
from astropy.io import fits
from astropy import table,coordinates
from astropy import units as u
from astropy.wcs.utils import skycoord_to_pixel
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import (ZScaleInterval, ImageNormalize)
from astropy.visualization import make_lupton_rgb
from astroquery.vizier import Vizier
from collections import Counter
from tensorflow import keras
from astropy.wcs import WCS,utils

## Want to include more candidates:
1. Make list of all candidates coordinates
2. Make list of all ring galaxies
3. Convert rad, dec, j2000 coords to x,y 
4. Get corresponding tiles
5. Create 200x200 pixels cutouts in u,r,z bands and store them in dataset
6. Visualize them and pick best

## 1. Gathering all known lenses: in currently used/labels/new lenses
- HSC-SSP: https://arxiv.org/abs/2107.07829                              from vizier
- HSC: https://ui.adsabs.harvard.edu/abs/2020A%26A...636A..87C/abstract  chan2020
- HSC: https://ui.adsabs.harvard.edu/abs/2020MNRAS.494.3156J/abstract    1 lens: HSC J0904–0102 : '09:04:29.75 −01:02:28.26'
- SDSS: https://ui.adsabs.harvard.edu/abs/2021MNRAS.502.4617T/abstract  talbot2021
- DESI: https://ui.adsabs.harvard.edu/abs/2021arXiv210900014R/abstract   rojas2021, only "sure lenses"
- CFIS papers :                                                          savary2021
- masterlens DB or new Swiss database ?

### Create new file with all new lens candidates: 'labels/new_lenses.csv'

In [129]:
label_dir = '../labels' 
keys = ['name', 'ra', 'dec', 'grade/score'] 

#chan
chan = open(label_dir + '/new lenses/chan2020.txt', 'r+')
lines = chan.read().split('\n')
chan.close()

chan_data = [line.split(' ')[:4] for line in lines[1:]]
chan_df = pd.DataFrame(chan_data, columns = keys)

#rojas
rojas = open(label_dir + '/new lenses/rojas2021.txt', 'r+')
lines = rojas.read().split('\n')
rojas.close()

rojas_data = [line.split(' ')[1:5] for line in lines[1:]]
rojas_df = pd.DataFrame(rojas_data, columns = keys)

#savary
savary = open(label_dir + '/new lenses/savary2021.txt', 'r+')
lines = savary.read().split('\n')
savary.close()

savary_data = [line.split(' ')[1:5] for line in lines[1:]]
savary_df = pd.DataFrame(savary_data, columns = keys)

#Jaelani (1 lens)
ra_dec = coordinates.SkyCoord('09:04:29.75 −01:02:28.26', unit=(u.hourangle, u.deg))
jae_data = [('HSCJ0904–0102', ra_dec.ra.deg, ra_dec.dec.deg, 3.0)]

In [131]:
#canameras lenses
lenses_cana = Vizier.get_catalogs(catalog="J/A+A/644/A163")[0]

#extracting relevent columns

cana_names = list(lenses_cana['Name'])#,'RAJ2000','DEJ2000','Grade']]
cana_ra = np.array(list(lenses_cana['RAJ2000']))
cana_dec = np.array(list(lenses_cana['DEJ2000']))
cana_grade = list(lenses_cana['Grade'])

#converting to ra&dec
ra_dec_degrees = [coordinates.SkyCoord( (cana_ra[i] + ' ' + cana_dec[i]),unit=(u.hourangle, u.deg)) for i in range(0, len(cana_ra))]
cana_ra_deg= [ coord.ra.deg  for coord in ra_dec_degrees]
cana_dec_deg= [ coord.dec.deg for coord in ra_dec_degrees]

#
cana_data = [ (cana_names[i], cana_ra_deg[i], cana_dec_deg[i], cana_grade[i]) for i in range(len(cana_names)) ]


jae_cana_df = pd.DataFrame(cana_data+jae_data, columns = keys)

In [132]:
new_cand_dfs = [chan_df, rojas_df, savary_df, jae_cana_df]
all_new_lenses = pd.concat(new_cand_dfs)

#save new lenses to csv
all_new_lenses.to_csv('../labels/new_lenses.csv')

### Combine with previous candidates (ra_dec.csv) and get tile

In [189]:
keys = ['name', 'ra', 'dec', 'grade/score' , 'X', 'Y'] 
new_list = pd.read_csv('../labels/new_lenses.csv')[keys[:-2]]
old_list = pd.read_csv('../labels/ra_dec.csv')

old_with_xy = pd.DataFrame([ ['previous', old_list['ra'][i],old_list['dec'][i], '--', 'tbd', 'tbd' ] for i in range(len(old_list))], columns = keys)
new_with_xy = pd.DataFrame([ [new_list['name'][i], new_list['ra'][i], new_list['dec'][i], new_list['grade/score'][i], 'tbd', 'tbd' ] for i in range(len(new_list))], columns = keys)

#save in file
all_lenses = pd.concat([new_with_xy, old_with_xy])
all_lenses.to_csv('../labels/all_lenses.csv')

## 2. Add ring galaxies: in currently used/labels/new lenses/rings
- PanSTARRS: https://ui.adsabs.harvard.edu/abs/2017ApJS..231....2T/abstract 

    185 Ring Galaxy Candidates Identified Automatically
    
    18 Collisional Ring Galaxies from (Madore et al. 2009) That Are Inside the Footprint of PanSTARRS DR1
    
- SDSS: https://ui.adsabs.harvard.edu/abs/2020MNRAS.491.3767S/abstract

    443 Ring galaxies in SDSS
    
- DESI (same paper as in 1) <--can't find the cat

In [ ]:
ring1 = './labels/rings/ringsPS1.txt'
ring1 = './labels/rings/ringsSDSS.txt'